In [10]:
import multiprocessing as mp
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [11]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'


In [12]:
tagg = [ '거래금액','건축년도','년','도로명','도로명건물본번호코드','도로명건물부번호코드',
         '도로명시군구코드','도로명일련번호코드','도로명지상지하코드','도로명코드','법정동','법정동본번코드',
         '법정동부번코드','법정동시군구코드','법정동읍면동코드','법정동지번코드','아파트',
         '월','일','일련번호','전용면적','지번','지역코드','층','해제사유발생일','해제여부']

In [13]:
base_url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=OSuNRoNLvTbcIjbHj3pWw2WHibEa%2FhYBmhMnXocnaw6O%2Bg36NybrbO5jp38r%2BI646%2Fk9NLJoIVHRPWyKz%2Foiow%3D%3D&pageNo=1&numOfRows=10000&LAWD_CD={}&DEAL_YMD={}'

In [14]:
region_code = pd.read_csv('C:/Users/shic/Desktop/crawler_project/data_save/estate_trade/region_code_parsed.csv')
region_code = region_code[region_code['폐지여부'] != '폐지']

In [15]:
date_list = []
for year in range(2010,2022):
    for month in range(1,13):
        if year == 2021 and month == 7:
            break
        
        if len(str(month)) == 1:
            date_string = str(year) + '0' + str(month)
        else:
            date_string = str(year) + str(month)
        
        date_list.append(date_string)

In [8]:
urls_list[0:10]

['http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=OSuNRoNLvTbcIjbHj3pWw2WHibEa%2FhYBmhMnXocnaw6O%2Bg36NybrbO5jp38r%2BI646%2Fk9NLJoIVHRPWyKz%2Foiow%3D%3D&pageNo=1&numOfRows=10000&LAWD_CD=11000&DEAL_YMD=201001',
 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=OSuNRoNLvTbcIjbHj3pWw2WHibEa%2FhYBmhMnXocnaw6O%2Bg36NybrbO5jp38r%2BI646%2Fk9NLJoIVHRPWyKz%2Foiow%3D%3D&pageNo=1&numOfRows=10000&LAWD_CD=11110&DEAL_YMD=201001',
 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=OSuNRoNLvTbcIjbHj3pWw2WHibEa%2FhYBmhMnXocnaw6O%2Bg36NybrbO5jp38r%2BI646%2Fk9NLJoIVHRPWyKz%2Foiow%3D%3D&pageNo=1&numOfRows=10000&LAWD_CD=11140&DEAL_YMD=201001',
 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=OSuNRoNLvTbcIjbHj3pWw2WHibEa%2FhYBmhMnXocnaw6O%2Bg36

In [38]:
driver = webdriver.Chrome(dir_for_shinhan)

In [ ]:
df_list = []

for region in region_code.iterrows():
    for date in date_list:
        tmp_url = base_url.format(region[1]['법정동코드_cut'],date)
        driver.get(tmp_url)

        soup = BeautifulSoup(driver.page_source,'xml')
        total_row = soup.find('totalCount').text

        if total_row == '0':
            pass

        else:
            return_list = []
            for soup_parser in soup.find_all('item'):
                data_parser = re.sub('\<(.*?)\>',' ',str(soup_parser))
                data_parser = re.sub('\s{2,}',' ',data_parser).strip().split(' ')

                if len(data_parser) == 24:
                    data_parser = data_parser + ['0','0']

                add_df = pd.DataFrame(dict(zip(tagg,data_parser)),index=[0])
                add_df['법정동명'] = region[1]['법정동명']
                add_df['법정동코드_cut'] = region[1]['법정동코드_cut']

                return_list.append(add_df)

            df_list.append(pd.concat(return_list,axis=0))